In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import os, re
import nltk
BASE_DIR = '../input/'
LABELED_TRAIN_DF = BASE_DIR + 'labeled_train_clean_reviews.csv'
TEST_DF = BASE_DIR + 'test_clean_reviews.csv'

In [2]:
labeled_train = pd.read_csv(LABELED_TRAIN_DF, header = 0)
test = pd.read_csv(TEST_DF, header = 0)
labeled_train["review"] = labeled_train["review"].astype(str)
test["review"] = test["review"].astype(str)
print "Read %d labeled train reviews" \
          "and %d test reviews" % (labeled_train["review"].size, test["review"].size)

Read 25000 labeled train reviewsand 25000 test reviews


Data leakage

Check if test["sentiment"] is correct

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)

In [4]:
from gensim.models import Word2Vec
EMBEDDING_DIM = 300  # Word vector dimensionality
MIN_WORD_COUNT = 40  # Minimum word count. Kaggle set to 40, to avoid attaching too much importance to individual movie titles.
NUM_THREADS = 4  # Number of threads to run in parallel
CONTEXT = 10  # Context window size
DOWNSAMPLING = 1e-3  # Downsample setting for frequent words
WORD2VEC_MODEL_FILE = BASE_DIR + \
    "word2vec_model_" + \
    str(EMBEDDING_DIM) + "dim_" + \
    str(MIN_WORD_COUNT) + "minwords_" + \
    str(CONTEXT) + "context"

word2vec_model = Word2Vec.load(WORD2VEC_MODEL_FILE)

# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
word2vec_model.init_sims(replace=True)

print("Number of words in the word2vec model vocabulary: %d" % len(word2vec_model.wv.vocab))

Number of words in the word2vec model vocabulary: 16490


Prepare

In [5]:
train_clean_reviews = labeled_train["review"].tolist()
test_clean_reviews = test["review"].tolist()

all_clean_reviews = train_clean_reviews + test_clean_reviews

In [6]:
print(np.random.choice(all_clean_reviews, 1))

['down periscope   has been in our library since it first arrived in vhs  since then  we have acquired the dvd and a digital from cinema now it is a quirky flick that does not go militarily overboard as either pro or con  it is first and foremost a comedy and as a vehicle for the main characters  i am quite surprised that a sequel has never been offered the movie has gained a following that borders on a cult obsession  even among the very young  i became aware of this while visiting the uss drum in mobile  alabama in  two  zero  zero  two   a group of cub scouts  my grandson among them  had all taken up the roles from the movie and planned to relive it during their overnighter on board it is a fun romp that makes you proud both of our navy and hollywood    which is rare company thanks to kelsey grammar  lauren holly and rob schneider for making what could have been an otherwise unremarkable movie  such great entertainment']


In [7]:
# We vectorize the text corpus by turning each text into a sequence of integers
# Each integer is the index of a token in the dictionary
from keras.preprocessing.text import Tokenizer

MAX_NUM_WORDS_FOR_KERAS_TOKENIZER = 200000
#
# num_words: the maximum number of words to keep, based on frequency.
keras_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS_FOR_KERAS_TOKENIZER)
#
# fit_on_texts accepts a list of strings, a generator of strings or 
# a list of list of strings. In the last case, it assumes each entry of the lists to be a token.
# Here we provide a list of strings.
keras_tokenizer.fit_on_texts(all_clean_reviews)
word_index = keras_tokenizer.word_index
print('Keras Tokenizer found %s unique tokens' % len(word_index))
#
# texts_to_sequences transforms each text in texts to a sequence of integers.
train_sequences = keras_tokenizer.texts_to_sequences(train_clean_reviews)
test_sequences = keras_tokenizer.texts_to_sequences(test_clean_reviews)

Using TensorFlow backend.


Keras Tokenizer found 101376 unique tokens


In [8]:
# We pad all text sequences to the same length.
# By default zeros are padded at the front.
from keras.preprocessing.sequence import pad_sequences

# Set max length for each review sequence.
MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN = 500

train_pad_sequences = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN)
test_pad_sequences = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN)

In [9]:
# Split train_sequences into train and validation. Ratio: 80/20
VALIDATION_SPLIT = 0.2
np.random.seed(1234)

# 
perm = np.random.permutation(len(train_sequences))
index_train = perm[:int(len(train_sequences)*(1-VALIDATION_SPLIT))]
index_val = perm[int(len(train_sequences)*(1-VALIDATION_SPLIT)):]

x_train = train_pad_sequences[index_train]
x_val = train_pad_sequences[index_val]
y_train = labeled_train["sentiment"][index_train].tolist()
y_val = labeled_train["sentiment"][index_val].tolist()

print('Randomly split %d pad sequences for training, %d for validation' % (len(x_train) ,len(x_val)))


Randomly split 20000 pad sequences for training, 5000 for validation


In [10]:
x_test = test_pad_sequences
y_test = test["sentiment"]

## Model Architecture

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, Conv1D, MaxPooling1D ,GlobalMaxPooling1D
from keras.initializers import Constant

In [16]:
def cnn_rnn_model(embed_dim=32, dropout_embed=0.2, dropout_cnn=0.5, dropout_rnn=0.2, rnn_units=32, num_filters=64, kernel_size=2):
    model = Sequential()
    
    LSTM_DROPOUT = 0.
    LSTM_RECCURENT_DROPOUT = 0.
    
    num_words = min(MAX_NUM_WORDS_FOR_KERAS_TOKENIZER, len(word_index) + 1)
    
    embedding_layer = Embedding(
            input_dim=num_words, # Size of the vocabulary, which is the number of unique tokens
            output_dim=embed_dim, # 300 is used for Word2Vec
            input_length=MAX_SEQUENCE_LENGTH_FOR_KERAS_RNN)
    output_layer = Dense(1, activation='sigmoid')
    
    model.add(embedding_layer)
    model.add(Dropout(dropout_embed))
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1))
    # Cannot use GlobalMaxPooling since you're feeding it into RNN
    model.add(MaxPooling1D(2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_cnn))
    model.add(Bidirectional(LSTM(rnn_units, 
                                 dropout=LSTM_DROPOUT, 
                                 recurrent_dropout=LSTM_RECCURENT_DROPOUT
                                )))
    model.add(Dropout(dropout_rnn))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [17]:
model = cnn_rnn_model(num_filters=32)

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           3244064   
_________________________________________________________________
dropout_4 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 499, 32)           2080      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 249, 32)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 249, 32)           1056      
_________________________________________________________________
dropout_5 (Dropout)          (None, 249, 32)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                16640     
__________

In [19]:
# batch_size: number of samples per gradient update
model.fit(x_train, y_train, batch_size=64, epochs=6, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/6
20000/20000 [==============================] - 287s 14ms/step - loss: 0.5522 - acc: 0.6906 - val_loss: 0.3363 - val_acc: 0.8624
Epoch 2/6
20000/20000 [==============================] - 283s 14ms/step - loss: 0.2488 - acc: 0.9050 - val_loss: 0.3129 - val_acc: 0.8814
Epoch 3/6
20000/20000 [==============================] - 283s 14ms/step - loss: 0.1375 - acc: 0.9538 - val_loss: 0.3150 - val_acc: 0.8910
Epoch 4/6
20000/20000 [==============================] - 283s 14ms/step - loss: 0.0746 - acc: 0.9776 - val_loss: 0.3317 - val_acc: 0.8816
Epoch 5/6
  512/20000 [..............................] - ETA: 4:09 - loss: 0.0235 - acc: 0.9961

KeyboardInterrupt: 

In [ ]:
y_test_pred = model.predict(x_test)

In [ ]:
y_test_pred_binary = map(lambda predict: 1 if predict > 0.5 else 0, y_test_pred)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

In [ ]:
print("The AUC score for CNN-BiLSTM model is : %.5f." %roc_auc_score(y_test, y_test_pred_binary))

In [ ]:
# Write the test results
output = pd.DataFrame(data={"id": test["id"], "sentiment": y_test_pred_binary})
output.to_csv(os.path.join('../', 'output', "cnn_bilstm_plain_embeddings.csv"), index=False, quoting=3)
print "Wrote to cnn_bilstm_plain_embeddings.csv"